In [4]:
import ipywidgets as widgets, random, math, os, importlib
from ipyevents import Event
from IPython.display import clear_output
import elgamal, num_algs

importlib.reload(elgamal)


valid_widgets = []

error_output = widgets.Output()
display(error_output)


txt_P = widgets.Text(
    description='p:',
    placeholder = 'Prime number',
)
valid_P = widgets.Valid(
    value=True,
)
valid_widgets.append(valid_P)
display(widgets.Box([txt_P, valid_P]))
valid_P.layout.visibility = 'hidden'


drpdwn_G = widgets.Dropdown(
    description='g:',
)
btn_generate_roots = widgets.Button(
    description='Generate',
)
valid_G = widgets.Valid(
    value=True,
)
valid_G.layout.visibility = 'hidden'
display(widgets.Box([drpdwn_G, btn_generate_roots, valid_G]))
valid_widgets.append(valid_G)


txt_X = widgets.Text(
    description='x:',
    disabled=False,
    placeholder = 'x in {1, ..., p - 1}',
)
btn_choose_random_X = widgets.Button(
    description='Choose randomly',
)
valid_X = widgets.Valid(
    value=True,
)
valid_X.layout.visibility = 'hidden'
valid_widgets.append(valid_X)
display(widgets.Box([txt_X, btn_choose_random_X, valid_X]))


txt_K = widgets.Text(
    description='k:',
    disabled=False,
    placeholder = 'k in {1, ..., p - 1}, (k, p-1)=1',
)
btn_choose_random_K = widgets.Button(
    description='Choose randomly',
)
valid_K = widgets.Valid(
    value=True,
)
chbox_randK = widgets.Checkbox(
    value=False,
    description='random',
    indent=False
)
valid_K.layout.visibility = 'hidden'
valid_widgets.append(valid_K)
display(widgets.Box([txt_K, btn_choose_random_K, valid_K, chbox_randK]))


txtarea_source = widgets.Textarea(
    description='Source text:',
    disabled=True,
    layout=widgets.Layout(
        width='600px'
    )
)
btn_open = widgets.FileUpload(
    description='Open',
)
chbox_ciphered = widgets.Checkbox(
    value=False,
    description='encrypted',
    indent=False
)
display(widgets.Box([txtarea_source, btn_open, chbox_ciphered]))


txtarea_result = widgets.Textarea(
    description='Result text:',
    disabled=True,
    layout=widgets.Layout(
        width='600px'
    )
)
btn_save = widgets.FileUpload(
    description='Save',
)
display(widgets.Box([txtarea_result, btn_save]))


btn_encrypt = widgets.Button(
    description='Encrypt',
)
btn_decrypt = widgets.Button(
    description='Decrypt',
)
display(widgets.Box([btn_encrypt, btn_decrypt]))


def show_param_error(widget, message: str):
    if widget != None:
        widget.layout.visibility = 'visible'
        widget.value = False
    if message > '':
        with error_output:
            print(message)
        
        
def clear_param_error(widget):
    with error_output:
        clear_output()
    if widget != None:
        widget.value = True
        widget.layout.visibility = 'hidden'
       

def validate_P(change):
    clear_param_error(valid_P)
    if not ((txt_P.value.isdigit() or txt_P.value == '') and num_algs.is_prime(int(txt_P.value))):
        show_param_error(valid_P, 'Incorrect value of p')
    else:
        clear_param_error(valid_P)
txt_P.observe(validate_P, names='value')


def validate_X(change):
    clear_param_error(valid_X)
    if not ((txt_X.value.isdigit() or txt_X.value == '') and 1 <= int(txt_X.value) <= int(txt_P.value) - 1):
        show_param_error(valid_X, 'Incorrect value of x')
    else:
        clear_param_error(valid_X)
txt_X.observe(validate_X, names='value')


def validate_K(change):
    clear_param_error(valid_K)
    if not ((txt_K.value.isdigit() or txt_K.value == '') and 1 <= int(txt_K.value) <= int(txt_P.value) - 1 and \
            num_algs.get_gcd(int(txt_K.value), int(txt_P.value)-1)[0] == 1):
        show_param_error(valid_K, 'Incorrect value of k')
    else:
        clear_param_error(valid_K)
txt_K.observe(validate_K, names='value')


def btn_choose_random_X_on_click(change):
    if valid_P.value:
        if txt_P.value == '':
            show_param_error(valid_P, 'Fill the field')
            show_param_error(valid_X, '')
        else:
            clear_param_error(valid_X)
            txt_X.value = str(random.randrange(1, int(txt_P.value)))
    else:
        show_param_error(valid_X, '')
btn_choose_random_X.on_click(btn_choose_random_X_on_click)


def btn_choose_random_K_on_click(change):
    if valid_P.value:
        if txt_P.value == '':
            show_param_error(valid_P, 'Fill the field')
            show_param_error(valid_K, '')
        else:
            clear_param_error(valid_K)
            p = int(txt_P.value)
            used = set()
            while True:
                k = random.randrange(1, p)
                gcd, _, _ = num_algs.get_gcd(k, p - 1)
                if gcd not in used and gcd == 1:
                    txt_K.value = str(k)
                    break
                else:
                    used.add(k)
    else:
        show_param_error(valid_K, '')
btn_choose_random_K.on_click(btn_choose_random_K_on_click)


def btn_generate_roots_on_click(change):
    if valid_P.value:
        if txt_P.value == '':
            show_param_error(valid_P, 'Fill the field')
            show_param_error(valid_G, '')
        else:
            clear_param_error(valid_G)
            roots = num_algs.get_primitive_roots(int(txt_P.value))
            drpdwn_G.options = roots
    else:
        show_param_error(valid_G, '')
btn_generate_roots.on_click(btn_generate_roots_on_click)

            
def open_file(change):
    global opened_file, bytes_for_int
    opened_file = btn_open.value[0].name
    txtarea_source.value = ''
    file_len = os.stat(opened_file).st_size
    with open(opened_file, 'rb') as file:
        if chbox_ciphered.value:
            bytes_for_int = int.from_bytes(file.read(1))
        else:
            bytes_for_int = 1
            file.seek(0)
        if file_len // bytes_for_int  - 1 <= 100:
            s = file.read()
            for i in range(0, len(s), bytes_for_int):
                txtarea_source.value += str(int.from_bytes(s[i:i+bytes_for_int])) + ' '
        else:
            s = file.read()
            cipher_text = list()
            for j in range(0, len(s), bytes_for_int):
                cipher_text.append(int.from_bytes(s[j:j+bytes_for_int]))
            for i in range(50):
                txtarea_source.value += str(cipher_text[i]) + ' '
            txtarea_source.value += '... '
            for i in range(len(cipher_text) - 50, len(cipher_text), 1):
                txtarea_source.value += str(cipher_text[i]) + ' '
btn_open.observe(open_file, names='value')


def save_file(change):
    saved_file_name = btn_save.value[0].name
    with open(saved_file_name, 'wb') as saved_file:
        if chbox_ciphered.value:
            for byte in result_text:
                saved_file.write(byte.to_bytes())
        else:
            saved_file.write(bytes_for_int.to_bytes())
            for byte in result_text:
                saved_file.write(byte.to_bytes(bytes_for_int))
            with error_output:
                print("The file's saved")
btn_save.observe(save_file, names='value')
    
    
def btn_encrypt_on_click(change):
    valid = True
    global result_text
    global cipher_text
    for w in valid_widgets:
        if not w.value:
            valid = False
    if valid:
        clear_param_error(None)
        global bytes_for_int
        bytes_for_int = 1
        cipher_text = []
        txtarea_result.value = ''
        if txt_P.value != '' and drpdwn_G.value != '' and txt_X.value != '' and (txt_K.value != '' or chbox_randK.value) and txtarea_source.value != '':
            clear_param_error(None)
            p = int(txt_P.value)
            g = int(drpdwn_G.value)
            x = int(txt_X.value)
            y = num_algs.pow_mod(g, x, p)
            with error_output:
                print(f'y = {y}')
            for s in open(opened_file, 'rb'):
                for byte in s:
                    if p < byte:
                        clear_param_error(valid_P)
                        show_param_error(valid_P, 'M must be less that p')
                        return
                    k = random.randrange(1, p) if chbox_randK.value else int(txt_K.value)
                    cipher_pare = elgamal.encrypt(byte, p, g, y, k)
                    if cipher_pare[0] >= 256 or cipher_pare[1] >= 256:
                        bytes_for_int = 2
                    cipher_text.extend(cipher_pare)
            result_text = cipher_text
            if len(cipher_text) < 100:
                for byte in cipher_text:
                    txtarea_result.value += str(byte) + ' '
            else:
                for i in range(50):
                    txtarea_result.value += str(cipher_text[i]) + ' '
                txtarea_result.value += '... '
                for i in range(len(cipher_text) - 50, len(cipher_text)):
                    txtarea_result.value += str(cipher_text[i]) + ' '
        else:
            show_param_error(None, 'Fill the fields')
    else:
        show_param_error(None, 'Invalid values')
btn_encrypt.on_click(btn_encrypt_on_click)
        
    
def btn_decrypt_on_click(change):
    valid = True
    for w in valid_widgets:
        if not w.value:
            valid = False
    if valid:
        clear_param_error(None)
        global decipher_text, result_text, cipher_text, bytes_for_int
        if txt_X.value != '' and txt_P.value != '' and txtarea_source.value != '':
            clear_param_error(None)
            x = int(txt_X.value)
            p = int(txt_P.value)
            cipher_text = list()
            with open(opened_file, 'rb') as source_file:
                bytes_for_int = int.from_bytes(source_file.read(1))
                while ((byte := source_file.read(bytes_for_int)) != b''):
                    cipher_text.append(int.from_bytes(byte))
            a_list = cipher_text[::2]
            b_list = cipher_text[1::2]
            decipher_text = [elgamal.decrypt(a, b, x, p) for a, b in zip(a_list, b_list)]
            txtarea_result.value = ''
            if len(decipher_text) < 100:
                for byte in decipher_text:
                    txtarea_result.value += str(byte) + ' '
            else:
                for i in range(50):
                    txtarea_result.value += str(decipher_text[i]) + ' '
                txtarea_result.value += '... '
                for i in range(len(decipher_text) - 50, len(decipher_text)):
                    txtarea_result.value += str(decipher_text[i]) + ' '
            result_text = decipher_text
        else:
            show_param_error(None, 'Fill the fields')
    else: 
        show_param_error(None, 'Invalid values')
btn_decrypt.on_click(btn_decrypt_on_click)

Output()

Box(children=(Text(value='', description='p:', placeholder='Prime number'), Valid(value=True)))

Box(children=(Dropdown(description='g:', options=(), value=None), Button(description='Generate', style=ButtonS…

Box(children=(Text(value='', description='x:', placeholder='x in {1, ..., p - 1}'), Button(description='Choose…

Box(children=(Text(value='', description='k:', placeholder='k in {1, ..., p - 1}, (k, p-1)=1'), Button(descrip…

Box(children=(Textarea(value='', description='Source text:', disabled=True, layout=Layout(width='600px')), Fil…

Box(children=(Textarea(value='', description='Result text:', disabled=True, layout=Layout(width='600px')), Fil…

Box(children=(Button(description='Encrypt', style=ButtonStyle()), Button(description='Decrypt', style=ButtonSt…

In [3]:
import elgamal, ipywidgets as widgets, os, importlib
importlib.reload(elgamal)


error_output = widgets.Output()
display(error_output)


txt_P = widgets.Text(
    description='p:',
)
display(txt_P)


txt_G = widgets.Text(
    description='g:',
)
display(txt_G)


txt_Y = widgets.Text(
    description='y:',
)
display(txt_Y)


del txtarea_source
txtarea_source = widgets.Textarea(
    description='Source text:',
    disabled=True,
    layout=widgets.Layout(
        width='600px'
    )
)
btn_open = widgets.FileUpload(
    description='Open',
)
display(widgets.Box([txtarea_source, btn_open]))


txtarea_result = widgets.Textarea(
    description='Result text:',
    disabled=True,
    layout=widgets.Layout(
        width='600px'
    )
)
btn_save = widgets.FileUpload(
    description='Save',
)
display(widgets.Box([txtarea_result, btn_save]))


btn_break = widgets.Button(
    description='Break',
)
display(btn_break)


def open_file(change):
    global opened_file, bytes_for_int, cipher_text
    cipher_text = []
    opened_file = btn_open.value[0].name
    txtarea_source.value = ''
    file_len = os.stat(opened_file).st_size
    with open(opened_file, 'rb') as file:
        bytes_for_int = int.from_bytes(file.read(1))
        if file_len // bytes_for_int  - 1 <= 100:
            s = file.read()
            for i in range(0, len(s), bytes_for_int):
                cipher_text.append(int.from_bytes(s[i:i+bytes_for_int]))
                txtarea_source.value += str(int.from_bytes(s[i:i+bytes_for_int])) + ' '
        else:
            s = file.read()
            for j in range(0, len(s), bytes_for_int):
                cipher_text.append(int.from_bytes(s[j:j+bytes_for_int]))
            for i in range(50):
                txtarea_source.value += str(cipher_text[i]) + ' '
            txtarea_source.value += '... '
            for i in range(len(cipher_text) - 50, len(cipher_text), 1):
                txtarea_source.value += str(cipher_text[i]) + ' '
btn_open.observe(open_file, names='value')


def save_file(change):
    saved_file_name = btn_save.value[0].name
    with open(saved_file_name, 'wb') as saved_file:
        for byte in plain_text:
            saved_file.write(byte.to_bytes())
        with error_output:
            print("The file's saved")
btn_save.observe(save_file, names='value')


def btn_break_on_click(change):
    global plain_text
    file_extension = opened_file.split('.')[-1]
    plain_text = []
    p = int(txt_P.value)
    g = int(txt_G.value)
    y = int(txt_Y.value)
    plain_text = elgamal.break_cipher(cipher_text, file_extension, y, g, p)
    if len(plain_text) < 100:
        for byte in plain_text:
            txtarea_result.value += str(byte) + ' '
    else:
        for i in range(50):
            txtarea_result.value += str(plain_text[i]) + ' '
        txtarea_result.value += '... '
        for i in range(len(plain_text) - 50, len(plain_text)):
            txtarea_result.value += str(plain_text[i]) + ' '
btn_break.on_click(btn_break_on_click)


Output()

Text(value='', description='p:')

Text(value='', description='g:')

Text(value='', description='y:')

Box(children=(Textarea(value='', description='Source text:', disabled=True, layout=Layout(width='600px')), Fil…

Box(children=(Textarea(value='', description='Result text:', disabled=True, layout=Layout(width='600px')), Fil…

Button(description='Break', style=ButtonStyle())